## Hebbian Learning
if neuron x repeatedly triggers neuron y, the synaptic knob connecting x to y gets larger. 
$$ W_{xy} = W_{x} + \eta xy $$
where $ W_{xy}$ means the connection between x and y (weight), and $\eta$ means the learning rate.
However, the downside of the formula is the weight $W_{xy}$ is non-decending which will eventually diverge.

## Perceptron: Simplified Model
![title](asserts/perceptron.png)

Fire if the combined input exceeds the threshold.

$$ Y = \begin{cases}
1 & \text{if } \sum w_i x_i - T \ge 0 \\
0 & \text{else} 
\end{cases}
$$

Extend the equaltion into a learning algorithm as follows:
$$ w = w + \eta (d(x) - y(x))x $$
where we introduce a desired output $ d(x) $

Each perceptron can form OR, NOT, AND, but it can't form **XOR**, so it is **not** a universal machine.
![form_eqs](asserts/single_perceptron.png)

In this way, a multi-layer perceptron is needed. A multi-layer perceptron is a universal machine and compute any functions.
![multi-layer-perceptron](asserts/xor_perceptron.png)

## Perceptron with real inputs
Y is an affine (a linear function with a offset b) of X.
$$ \begin{align*}
y &= \theta\left(\sum w_i x_i) + b\right) \\ 
\theta (z) &= \begin{cases} 
1 & \text{if } \sum w_i x_i - T \ge 0 \\
0 & \text{else} 
\end{cases}
\end{align*}
$$

where $ b = -T $

In this way, boolean perceptrons can be linear classifiers.
![bool_split](asserts/boolean_split.png)


In [ ]:
from typing import List
# build a pategon spliter
# we need spliter for each edge (6 in total), and a and to identify wether the point is within the pategon or outside of the pategon
# perceptron is an affine function that takes in an input(the point position), and a weight, and output wether 1 or 0 to indicate 
# the boolean spliter.
class Perception:
    def __init__(self, weight: List[int], threshold: int) -> None:
        self.weight = weight
        self.threhold = threshold

    def run(self, p_input: List[int]) -> int:
        # return w1 * input1 + w2 * input2 + ...
        total = sum(wi * ii for wi, ii in zip(self.weight, p_input ))
        return 1 if total >= self.threhold else 0

# takes in a list of perceptrons as a 2D array where each inner list repersents the perceptrons in the same nn layers.
# input is a list of input to each 1st layer perceptrons.
def run_nn(perceptions: List[List[Perception]], inputs: List[int]):
    current_inputs = inputs
    # run each layer 
    for layer in perceptions:
        # run each node
        layer_output = []
        for input in inputs:
            # feed it input each Perception
            for p in layer:
                p_out = p.run(input)
                layer_output.append(p_out)
        current_inputs = layer_output
        print(current_inputs)

    print(current_inputs[0])

def test_one_perception():
   p1 = Perception(1,1) 
   p2 = Perception(1,1)
   p3 = Perception(1,2)
   nn = [[p1,p2],[p3]]
   inputs = [1,1]
   run_nn(nn,inputs)

test_one_perception()

[False, False, False, False]
[False, False]
False
